In [1]:
import pandas as pd
from rdkit import Chem
import seaborn as sns
from rdkit.Chem import QED, Draw, Descriptors, rdDepictor, rdMolDescriptors
from rdkit.Chem.FilterCatalog import FilterCatalogParams, FilterCatalog
from tqdm import tqdm
import matplotlib.pyplot as plt
rdDepictor.SetPreferCoordGen(True)
from rdkit.Contrib.SA_Score import sascorer

<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for class boost::shared_ptr<class RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.


In [2]:
params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
catalog = FilterCatalog(params)

def calculate_brenk_relative(mol):
    num_alerts = len(catalog.GetMatches(mol))
    num_fragments = mol.GetNumAtoms()

    if num_fragments == 0:
        return 0
    return num_alerts / num_fragments

In [3]:
import os
import subprocess
from rdkit import Chem
from rdkit.Chem import AllChem

def smiles_to_pdbqt(smiles: str, output_file: str = "legant.pdbqt"):
    """Конвертирует SMILES в PDBQT через Open Babel."""
    # Создание молекулы из SMILES и добавление водородов
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    status = AllChem.EmbedMolecule(mol)
    if status == -1:
        raise RuntimeError("Не удалось сгенерировать 3D-структуру")
    
    # Сохранение во временный файл .mol
    temp_mol = "temp.mol"
    Chem.MolToMolFile(mol, temp_mol)
    if not os.path.exists(temp_mol):
        raise FileNotFoundError("Временный файл не создан")
    
    # Конвертация в PDBQT через Open Babel
    cmd = f"obabel {temp_mol} -O legant.pdbqt"
    result = subprocess.run(
        cmd, 
        shell=True, 
        capture_output=True, 
        text=True
    )

    if result.returncode != 0:
        error_msg = f"Ошибка Open Babel:\n{result.stderr}"
        if "Invalid output format" in result.stderr:
            error_msg += "\nУбедитесь, что Open Babel установлен и добавлен в PATH"
        raise RuntimeError(error_msg)

    if not os.path.exists(output_file):
            raise FileNotFoundError(f"Файл {output_file} не создан")
    
    #os.remove(temp_mol)

def run_vina_docking(protein_pdbqt: str, ligand_pdbqt: str, center: tuple = (27.116, 24.090, 14.936), size: tuple = (10, 10, 10)) -> float:
    """Запускает докинг и возвращает энергию связывания."""
    # Создание конфигурационного файла для Vina
    config = f"""
    receptor = {protein_pdbqt}
    ligand = {ligand_pdbqt}
    out = result.pdbqt
    center_x = {center[0]}
    center_y = {center[1]}
    center_z = {center[2]}
    size_x = {size[0]}
    size_y = {size[1]}
    size_z = {size[2]}
    exhaustiveness = 16
    cpu = 12
    """
    with open("config.txt", "w") as f:
        f.write(config)
    
    # Запуск AutoDock Vina
    with open("log.txt", "w") as log_file:
        result = subprocess.run(
            "vina_1.2.7_win --config config.txt",
            stdout=log_file,
            text=True,
            check=True,
            shell=True
        )
        if result.returncode != 0:
            print("Ошибка Vina:", result.stderr)
            return None
    
    # Извлечение энергии связывания из лога
    with open("log.txt", "r") as f:
        log = f.read()
        affinity_values = []
        for line in log.split("\n"):
            if line.strip().startswith("1"):  # Первая строка с результатами
                parts = line.split()
                if len(parts) >= 2:
                    try:
                        affinity = float(parts[1])
                        affinity_values.append(affinity)
                    except ValueError:
                        continue
    
        # Возвращаем лучшую энергию
        if affinity_values:
            return min(affinity_values)
        else:
            print("Энергии связывания не найдены")
            return None

In [ ]:
import pickle
from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator

FP_CACHE = 'chembl_fps.pkl'
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=3,fpSize=2048)
    
# Загрузка предобработанных данных
try:
    with open(FP_CACHE, 'rb') as f:
        data = pickle.load(f)
        chembl_smiles = data['smiles']
        chembl_fps = data['fps']
    print(f"Успешно загружено {len(chembl_fps)} фингерпринтов")
except Exception as e:
    raise RuntimeError(f"Ошибка загрузки кэша: {str(e)}")

def fast_tanimoto(smiles: str) -> float:
    """Мгновенный расчет максимального сходства"""
    try:
        mol = Chem.MolFromSmiles(smiles)            
        query_fp = mfpgen.GetFingerprint(mol)
        similarities = DataStructs.BulkTanimotoSimilarity(query_fp, chembl_fps)
        max_sim = max(similarities)
        return max_sim*2 if max_sim < 0.3 else max_sim
        
    except Exception as e:
        print(f"Ошибка: {str(e)}")
        return 0.0

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from rdkit.Chem import rdEHTTools
from rdkit.Chem import AllChem

def predict_hsa_binding(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return {"error": "Invalid SMILES string"}

        # рассчет свойств
        sum = 0
        mw = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        aromatic_rings = Lipinski.NumAromaticRings(mol)
        carboxy_groups = len(mol.GetSubstructMatches(Chem.MolFromSmarts("C(=O)O")))

        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        _, res = rdEHTTools.RunMol(mol)
        Atoms = res.GetAtomicCharges()
        for a in Atoms:
          sum += a
        negative_charge = sum < 0
        
        heterocycles = Chem.MolFromSmarts("[!C;R]")
        has_heterocycle = mol.HasSubstructMatch(heterocycles)
        
        binding_info = {"Взаимодействие молекулы с HSA": "Низкая"}
        if (logp > 0 and logp < 3) and mw < 400:
            binding_info["Взаимодействие молекулы с HSA"] = "Средняя"
        if (aromatic_rings >= 1 and carboxy_groups >= 1 and negative_charge and mw < 500) or (has_heterocycle and negative_charge and mw > 250 and logp > 1):
            binding_info["Взаимодействие молекулы с HSA"] = "Высокая"
        return binding_info

    
    except Exception as e:
        return {"error": str(e)}

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, rdMolDescriptors
from joblib import dump, load
import numpy as np
import hyp

irritation_model = load('model_irrit.joblib')
melanin_model = load('model_melanin.joblib')
corneal_model = load('catboost_model_joblib.pkl')

def smiles_to_features(smiles:str):
    mol = Chem.MolFromSmiles(smiles)
    descriptor_names = [name for name, func in Descriptors.descList]
    def calc_descriptors(mol):
        if mol is not None:
            descriptor_values = [func(mol) for name, func in Descriptors.descList]
            return descriptor_values
        else:
            return [None] * len(descriptor_names)
    
    descriptor_values = calc_descriptors(mol)
    
    
    features = pd.Series(dict(zip(descriptor_names, descriptor_values)))
    
    return features

def get_fingerprint(molecule):
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=hyp.fingerprint_radius,fpSize=hyp.fingerprint_length)
    if molecule is None:
        return np.zeros((hyp.fingerprint_length,))
    fingerprint = mfpgen.GetFingerprint(molecule)
    return np.array(fingerprint)    

def calculate_properties(smiles):
    """Вычисление всех свойств для одной молекулы"""
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None

    sa_score = sascorer.calculateScore(mol)
    qed_value = QED.qed(mol) 
    brenk_score = calculate_brenk_relative(mol)
    tonimoto = fast_tanimoto(smiles)
    try:
        smiles_to_pdbqt(smiles)
        docking = run_vina_docking("COX-2.pdbqt", "legant.pdbqt")
    except:
        docking = None
    
    properties = {
        'smiles': smiles,
        'molecule': mol,
        'binding_energy': docking,
        'sascore': sa_score,
        'qed': qed_value,
        'brenk': brenk_score,
        'hsa': predict_hsa_binding(smiles),
        'tonimoto': tonimoto,
        'mol_weight': Descriptors.MolWt(mol),
        'logp': corneal_model.predict(smiles_to_features(smiles)), 
        'melanin': melanin_model.predict_proba(np.expand_dims(get_fingerprint(mol), axis=0))[0, 1],
        'irrit': irritation_model.predict_proba(np.expand_dims(get_fingerprint(mol), axis=0))[0, 1], 
        'tpsa': rdMolDescriptors.CalcTPSA(mol),
        'rotatable_bonds': Descriptors.NumRotatableBonds(mol)
    }
    return properties

In [ ]:
calculate_properties('CC(=O)c1cc(OCCN)c(C)cc1C1=C(C)C(=O)COC=C1')

In [ ]:
def filter_molecules(df):
    """Фильтрация молекул по заданным критериям"""
    mask1 = df['binding_energy'] < -6
    mask2 = df['sascore'] < 4.5
    mask3 = df['qed'] > 0.6
    mask4 = df['brenk'] < 0.4

    condition_count = mask1.astype(int) + mask2.astype(int) + mask3.astype(int) + mask4.astype(int)
    filtered = df[condition_count >= 3]
    
    return filtered.sort_values(by=['qed', 'sascore'], ascending=[False, True])

In [ ]:
smiles_list = [
    'N=C=C=C(C=NC=C=C=CC=CO)Nn1ncooo1',
    'C=C=C=CC(C(=O)CC)C1OC(=C)N1O',
    'C=C=C1C(O)CC1C(C)CCCC',
    'CCCC1(OO)CC1C',
    'COC(=O)C1C(=O)C2=C(OC)c3c2[nH]c(C)c31',
    'CCOCC(C)C(=O)O',
    'CCCCC1(N)C(=O)C1C',
    'CC(=O)CCC1=NC(O)=C1O',
    'CCCOc1cc(C(C)=O)c(-c2c(N)c(=O)c3oc(=O)c2=3)cc1C',
    'CC(=O)c1cc(OCCN)c(C)cc1C1=C(C)C(=O)C2=C1CO2',
    'CC(=O)c1cc(OCCN)c(C)cc1C1=C(C)C(=O)COCC1',
    'CC(=O)c1cc(OCCN)c(C)cc1CCC1=CCC(=O)CO1',
    'CC(=O)c1cc(OCCN)c(C)cc1C1=C(C)C(=O)COC=C1',
]

In [ ]:
results = []
for smiles in tqdm(smiles_list, desc='Processing molecules'):
    props = calculate_properties(smiles)
    if props:
        results.append(props)

In [ ]:
df = pd.DataFrame(results)
filtered_df = filter_molecules(df).head(10)

In [ ]:
filtered_df.head(20)

In [ ]:
print("Top molecules after filtering:")
display(filtered_df[['smiles', 'qed', 'sascore', 'brenk', 'tonimoto','binding_energy']].head())

In [ ]:
mols = filtered_df['molecule'].tolist()
img = Draw.MolsToGridImage(mols, molsPerRow=5, subImgSize=(200,200))
display(img)